# Passive API prototyping

In [1]:
%matplotlib nbagg

In [2]:
import ipywidgets

In [3]:
import os

In [4]:
os.getpid()

658969

In [5]:
# Thx Mat!
def notebook_fullwidth():
    from IPython.display import display, HTML
    """Set notebook display to 95% fullwidth for more available space"""
    display(HTML(data="""
    <style>
        div#notebook-container    { width: 95%; }
        div#menubar-container     { width: 65%; }
        div#maintoolbar-container { width: 99%; }
    </style>
    """))
notebook_fullwidth()

In [6]:
%env OTEL_ENABLE=1
%env OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317
%env LIBERTEM_DECTRIS_LOG_LEVEL=debug
%env OMP_NUM_THREADS=1

env: OTEL_ENABLE=1
env: OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317
env: LIBERTEM_DECTRIS_LOG_LEVEL=debug
env: OMP_NUM_THREADS=1


In [7]:
from matplotlib import pyplot as plt

In [8]:
import time

import numpy as np

from libertem_live.detectors.dectris.acquisition import DectrisAcquisition, DectrisDetectorConnection
from libertem_live.detectors.dectris.DEigerClient import DEigerClient
from libertem_dectris import DectrisConnection
from libertem.executor.pipelined import PipelinedExecutor
from libertem.executor.inline import InlineJobExecutor
from libertem_live.api import LiveContext
from libertem.api import Context
from libertem.udf.sum import SumUDF
from libertem.viz.bqp import BQLive2DPlot
from libertem.udf.sumsigudf import SumSigUDF
from libertem.udf.sum import SumUDF
from libertem.udf.masks import ApplyMasksUDF

In [9]:
from libertem.udf import UDF
from libertem.udf.base import NoOpUDF

In [10]:
from libertem.common.tracing import maybe_setup_tracing

In [11]:
maybe_setup_tracing(service_name="notebook-main")

In [12]:
executor = PipelinedExecutor(
    spec=PipelinedExecutor.make_spec(
        cpus=range(14), cudas=[0]
    ),
    pin_workers=True,
    delayed_gc=False,
)

In [13]:
ctx = LiveContext(executor=executor, plot_class=BQLive2DPlot)

In [14]:
def trigger(aq):
    pass

In [15]:
placeholder = ipywidgets.Output()
placeholder

Output()

In [16]:
conn = DectrisDetectorConnection(
    data_host="127.0.0.1",
    data_port=9999,
    api_host="127.0.0.1",
    api_port=8910,
    frame_stack_size=24,
    num_slots=4000,
    bytes_per_frame=18000, # int(134*512/4),  # approx.
    huge_pages=True,
)

try:
    while True:
        pending_aq = conn.wait_for_acquisition(3600)
        if pending_aq is None:  # timeout
            continue

        print(f"got pending acquisition: {pending_aq}")
        import math
        side_length = int(math.sqrt(pending_aq.detector_config.get_num_frames()))

        # create an acquisition from a pending acquisition:
        aq = ctx.prepare_acquisition(
            'dectris',
            conn=conn,
            pending_aq=pending_aq,

            nav_shape=(side_length, side_length),

            trigger=trigger,
            #name_pattern="$id-from-libertem",

            # this can't be too large, as that will overflow the shared memory
            # (too much data is sent to a single process, which can't keep up in decompression and computation)
            # this can't be too small, as you'll get per-partition overheads
            frames_per_partition=1024,
        )

        udf = SumSigUDF()

        placeholder.clear_output()
        with placeholder:
            plot = BQLive2DPlot(channel='intensity', min_delta=0.2, dataset=aq, udf=udf)
            plot.display()

        t0 = time.perf_counter()
        ress = [ctx.run_udf(dataset=aq, udf=udf, plots=[plot])]
        t1 = time.perf_counter()
        print(f"elapsed: {t1-t0}")
        conn.log_stats()
finally:
    conn.close()

[2023-01-27T14:40:29Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECT_DELAYED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:29Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECTED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:29Z INFO  libertem_dectris::common] monitoring DectrisReceiver: HANDSHAKE_SUCCEEDED @ tcp://127.0.0.1:9999


got pending acquisition: <DectrisPendingAcquisition series=1134 config=DetectorConfig { ntrigger: 1048576, nimages: 1, trigger_mode: EXTS, x_pixels_in_detector: 96, y_pixels_in_detector: 96, bit_depth_image: 16 }>


[2023-01-27T14:40:32Z DEBUG libertem_dectris::dectris_py] dheader: DHeader { htype: "dheader-1.0", header_detail: "basic", series: 1134 }
[2023-01-27T14:40:37Z INFO  libertem_dectris::dectris_py] done in 5.403885793s, reading acquisition footer...
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: DISCONNECTED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECT_RETRIED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECT_DELAYED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CLOSED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECT_RETRIED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_dectris::common] monitoring DectrisReceiver: CONNECT_DELAYED @ tcp://127.0.0.1:9999
[2023-01-27T14:40:37Z INFO  libertem_d

PanicException: could not join background thread!: Any { .. }